# LSST Filters

This notebook explores the LSST filter set and the fiducial LSST atmosphere


In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, '../')

import numpy as np
import sncosmo
from astropy.table import Table
from matplotlib import pyplot as plt
from scipy.stats import binned_statistic
from sn_analysis import filters
from pwv_kpno import pwv_atm


In [ ]:
fig_dir = Path('.') / 'notebook_figs' / 'lsst_filters'
fig_dir.mkdir(exist_ok=True, parents=True)

# Make sncosmo aware of the LSST filter set
filters.register_lsst_filters(force=True)


The LSST filter set is provided in a way that allows us to separate the hardware and atmospheric contributions to the overall bandpass response function. This is useful because it allows us to add out own customized atmospheric component.

In [ ]:
def plot_lsst_filters():
    """Plot the response curves for the LLST hardware and filters"""
    
    fig, (left_ax, right_ax) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(12, 6))

    # Transmissions that are NOT reported on a per band basis
    detector = sncosmo.get_bandpass('lsst_detector')
    lenses = sncosmo.get_bandpass('lsst_lenses')
    mirror = sncosmo.get_bandpass('lsst_mirrors')
    atm = sncosmo.get_bandpass('lsst_atmos_std')
    
    # Plot hardware transmissions on left axis
    left_ax.plot(detector.wave, detector.trans, label='Detector')
    left_ax.plot(lenses.wave, lenses.trans, label='Lenses')
    left_ax.plot(mirror.wave, mirror.trans, label='Mirrors')
    
    # Non hardware goes on the right axis
    right_ax.plot(atm.wave, atm.trans, color='C0', label='Atm.')
    
    # Band specific trnamsissions
    for i, band_letter in enumerate('ugrizy'):
        band_total = sncosmo.get_bandpass(f'lsst_total_{band_letter}')
        hardware = sncosmo.get_bandpass(f'lsst_hardware_{band_letter}')
        filter_only = sncosmo.get_bandpass(f'lsst_filter_{band_letter}')
        
        left_ax.fill_between(filter_only.wave, filter_only.trans, color=f'C{i}', alpha=.2, label='Filters' if i == 0 else None)
        left_ax.plot(hardware.wave, hardware.trans, color='k', label='All Hardware' if i == 0 else None)
        
        right_ax.plot(hardware.wave, hardware.trans, color='k', label='Hardware' if i == 0 else None)
        right_ax.fill_between(band_total.wave, band_total.trans, alpha=.2, label='Bandpass' if i == 0 else None)
        right_ax.plot(band_total.wave, hardware(band_total.wave) * atm(band_total.wave))
        
    left_ax.set_xlabel(r'Wavelength ($\AA$)')
    left_ax.set_ylabel('Transmission')
    left_ax.set_xlim(3000, 11_000)
    left_ax.set_ylim(0, 1)
    left_ax.set_title('Hardware')
    left_ax.legend(loc='lower right', framealpha=1)
    
    right_ax.set_xlabel(r'Wavelength ($\AA$)')
    right_ax.set_title('Bandpasses')
    right_ax.legend(loc='upper left', framealpha=1)
    

In [ ]:
plot_lsst_filters()
plt.savefig(fig_dir / 'filters_and_hardware.pdf')
plt.show()


Since we are sometimes interested in the specific effects of PWV absorption, we compare the reddest (largest) PWV absorption feature of the LSST standard atmosphere against our own atmospheric model (`pwv_kpno`). We compare the LSST atmosphere against PWV absorption for 4mm PWV binned at a 5 angstrom resolution.

In [ ]:
def fit_lsst_atm_continua(deg, *wave_ranges, lsst_atm='lsst_atmos_std'):
    """Fit the LSST atmosphere with a polynomial
    
    Args:
        deg                (int): The degree of the fitted polynomial
        *wave_ranges     (tuple): Start and end wavlengths of regions to fit
        lsst_atm (str, Bandpass): The atmosphere to fit
        
    Returns:
        - A tuple of fit parameters
        - A function representing the fitted polynomial
    """

    # Ensure the lsst_atm variable is a Bandpass object
    lsst_atm = sncosmo.get_bandpass(lsst_atm)
    
    # Fitt the LSST transmission over the given wavelength ranges
    wavelengths = np.concatenate([np.arange(*w) for w in wave_ranges])
    fit_params = np.polyfit(wavelengths, lsst_atm(wavelengths), deg)

    return fit_params, np.poly1d(fit_params)


def trans_for_pwv(pwv, wavelengths, resolution):
    """Retrieve the pwv_kpno transmission at given wavelengths and resolution

    Args:
        pwv           (float): The PWV along line of sight
        wavelengths (ndarray): Array of wavelengths to evaluate transmission at
        resolution    (float): Resolution to bin transmission at

    Returns:
        An array of transmission values
    """

    # Create bins that uniformly sample the given wavelength range
    # at the given resolution
    half_res = resolution / 2
    bins = np.arange(
        min(wavelengths) - half_res, 
        max(wavelengths) + half_res + resolution, 
        resolution)
    
    # Bin the atm model to the desired resolution
    atm_model = pwv_atm.trans_for_pwv(pwv)
    statistic_left, bin_edges_left, _ = binned_statistic(
        atm_model['wavelength'],
        atm_model['transmission'],
        statistic='mean',
        bins=bins[:-1]
    )
    
    statistic_right, bin_edges_right, _ = binned_statistic(
        atm_model['wavelength'],
        atm_model['transmission'],
        statistic='mean',
        bins=bins[1:]
    )

    statistic = (statistic_right + statistic_left) / 2
    
    dx = atm_model['wavelength'][1] - atm_model['wavelength'][0]
    bin_centers = bin_edges_left[:-1] + dx / 2
    
    # Evaluate the transmission at the desired wavelengths
    return np.interp(wavelengths, bin_centers, statistic)



In [ ]:
def plot_lsst_atm():
    """Plot the LSST standard atmosphere"""
    
    # Fit the LSST atmospheric continuum (i.e. the non-PWV transmission)
    lsst_atm = sncosmo.get_bandpass('lsst_atmos_std')
    continuum_wavelengths = [(8400, 8850), (10_000, 10_500)]
    fit_params, fit_func = fit_lsst_atm_continua(2, *continuum_wavelengths, lsst_atm=lsst_atm)
    fit_label = fr'{fit_params[0]: .1e} x$^2$ + {fit_params[1]: .1e} x + {fit_params[2]: .2f}'

    fig, (left_ax, right_ax) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(12, 6))
    
    # Plot fit to the continuum in left axis
    left_ax.plot(lsst_atm.wave, lsst_atm.trans, label='LSST Fiducial Atmosphere')
    left_ax.plot(lsst_atm.wave, fit_func(lsst_atm.wave), label=fit_label)
    for i, wave_range in enumerate(continuum_wavelengths):
        label = 'Continuum' if i == 0 else None
        left_ax.axvspan(*wave_range, color='C3', alpha=.1, label=label)
        
    left_ax.set_xlabel(r'Wavelength ($\AA$)')
    left_ax.set_ylabel('Total Transmission')
    left_ax.set_xlim(8000, 11000)
    left_ax.set_ylim(.5, 1)
    left_ax.legend(loc='lower left', framealpha=1)
    
    # Plot comparison of normalized and modeled absorption feature
    normalized_trans = lsst_atm.trans / fit_func(lsst_atm.wave)
    modeled_trans = trans_for_pwv(4, lsst_atm.wave, 5)
    
    right_ax.plot(lsst_atm.wave, modeled_trans, color='grey', label='pwv_kpno')
    right_ax.plot(lsst_atm.wave, normalized_trans, label='Normalized LSST')
    
    right_ax.set_xlabel(r'Wavelength ($\AA$)')
    right_ax.set_ylabel('PWV Transmission')
    right_ax.legend(loc='lower left', framealpha=1)


In [ ]:
plot_lsst_atm()
plt.savefig(fig_dir / 'atmospheric_continuum_fit.pdf')
plt.show()


A rough minimization shows that our choice of resolution and PWV concentration in the above plot were reasonable. 

In [ ]:
from scipy.optimize import minimize

fit_params, fit_func = fit_lsst_atm_continua(2, (8400, 8850), (10_000, 10_500))

lsst_atm = sncosmo.get_bandpass('lsst_atmos_std')
lsst_atm_normalized = sncosmo.Bandpass(
    wave=lsst_atm.wave, 
    trans=lsst_atm.trans / fit_func(lsst_atm.wave)
)


def summed_resids(x):
    pwv, res = x
    
    wavelengths = np.arange(8400, 10_500)
    modeled_trans = trans_for_pwv(pwv, wavelengths, res)
    normalized_trans = lsst_atm_normalized(wavelengths)
    stat = np.average(normalized_trans - modeled_trans)
    return abs(stat)

x0 = 4, 5
res = minimize(summed_resids, x0, bounds=[(2, 10), (1, 10)], tol=.01)
res.x
